In [410]:
import gym, time, torch, os, sys, importlib
import matplotlib.pyplot as plt
import numpy as np

from torch import nn, optim

sys.path.append(os.path.abspath('..')) # Add parent directory to path
import ppo
importlib.reload(ppo) # Prevents caching issues

from ppo import PPONetwork, PPOWrapper
from ppo import parameter_sweeper, pick_parameter_sweep_results, plot_sweep_results

In [411]:
# Initialize environment with gym_kwargs
gym_kwargs = {"id": "LunarLander-v2", "max_episode_steps": 1000}
env = gym.make(**gym_kwargs)

state, info = env.reset()
print("State: ", state)
print("Info: ", info)

observation_space = env.observation_space.shape[0]
action_space = env.action_space.n

print("Observation space: ", observation_space)
print("Action space: ", action_space)

a=0
state, reward, done, truncated, info = env.step(a)

print(f"Action: {a}, Reward: {reward}, Done: {done}, Truncated: {truncated}, Info: {info}")


State:  [ 0.00717831  1.4166478   0.72707343  0.2545496  -0.00831112 -0.16469298
  0.          0.        ]
Info:  {}
Observation space:  8
Action space:  4
Action: 0, Reward: -0.42988927258062404, Done: False, Truncated: False, Info: {}


/Users/axelsolhall/miniconda3/envs/PPOgym/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [412]:
# Initialize network
input_dim = observation_space
output_dim = action_space
hidden_dims = [256, 256, 128]
policy_hidden_dims = [64, 64, 32]
value_hidden_dims = [64, 64, 32]

network_kwargs = {"input_dim": input_dim, "output_dim": output_dim, "hidden_dims": hidden_dims, "policy_hidden_dims": policy_hidden_dims, "value_hidden_dims": value_hidden_dims}
network = PPONetwork(**network_kwargs)

In [ ]:
# Initialize PPO wrapper
gamma = 0.99
lam = 0.95
clip_epsilon = 0.2
initial_lr = 5e-4 # 5e-4 worked best so far
final_lr = 1e-5
value_coef = 1
entropy_coef = 0.05
batch_epochs = 1
batch_size = 512
batch_shuffle = True
checkpointing = True
anneal_clip = True
anneal_entropy = False
anneal_lambda = False

truncated_reward = -50

# Initialize PPO wrapper
ppo_kwargs = {"gamma": gamma, "lam": lam, "clip_epsilon": clip_epsilon, "initial_lr": initial_lr, "final_lr": final_lr, "value_coef": value_coef, "entropy_coef": entropy_coef, "batch_size": batch_size, "batch_epochs": batch_epochs, "batch_shuffle": batch_shuffle, "checkpointing": checkpointing, "anneal_clip": anneal_clip, "anneal_entropy": anneal_entropy, "anneal_lambda": anneal_lambda, "truncated_reward": truncated_reward}
ppo = PPOWrapper(env, network, **ppo_kwargs)

# Make sure it works
_ = ppo.train_model(100000, display=True)

	 - Env 0: reward = -110.21, steps taken = 8.0
	 - Env 1: reward = -134.48, steps taken = 48.0
	 - Env 2: reward = -111.81, steps taken = 22.0
	 - Env 3: reward = -99.35, steps taken = 34.0
	 - Env 4: reward = -13.96, steps taken = 16.0
	 - Env 5: reward = -163.52, steps taken = 27.0
	 - Env 6: reward = -210.17, steps taken = 100.0
	 - Env 7: reward = -188.57, steps taken = 60.0
	 - Env 8: reward = -213.83, steps taken = 148.0
	 - Env 9: reward = 0.25, steps taken = 53.0
Episode: 0	 avg life: 86.78, eval reward: -124.56, eval ma: -12.46
	 - Env 0: reward = -105.26, steps taken = 9.0
	 - Env 1: reward = -163.67, steps taken = 66.0
	 - Env 2: reward = -165.31, steps taken = 58.0
	 - Env 3: reward = -214.04, steps taken = 74.0
	 - Env 4: reward = -117.42, steps taken = 106.0
	 - Env 5: reward = -132.96, steps taken = 49.0
	 - Env 6: reward = -131.46, steps taken = 22.0
	 - Env 7: reward = -118.00, steps taken = 15.0
	 - Env 8: reward = -190.45, steps taken = 100.0
	 - Env 9: reward = -171